In [1]:
%matplotlib inline 
from collections import OrderedDict
import numpy as np
import pandas as pd

def iter_prefixes(t, i): 
    "from longest to shortest"

    for j in reversed(range(i+1, len(t) + 1)):
        yield t[i:j]


def ms(t, s, i): 
    """the longest prefix of t[i:] that occurs in s"""
    for prefix in iter_prefixes(t, i): 
        #print s + ": " + prefix, 
        if prefix in s:
            #print "*"
            return len(prefix)
        #print
    return 0


def bwt(s):
    from collections import Counter
    s = s + "#" 
    A = np.array([list(s[i:] + s[:i]) for i in range(len(s))])
    As = np.vstack(sorted(A, key=tuple))
    CF = Counter(As[:, 0]) 
    S = sorted(CF)
    C = [0] * (len(CF) + 1)

    for i in range(len(S)):
        if i == 0:
            continue
        C[i] = C[i-1] + CF[S[i-1]]
    C[i+1] = len(s)
    return "".join(As[:, len(s) - 1]), "".join(S), "-".join(map(str, C)), As

def table1(t, s):
    a = pd.DataFrame(OrderedDict([('i', [-1] + range(len(t))),
                                  ('t_i', [''] + list(t)), 
                                  ('MS', [1] + map(lambda i: ms(t, s, i), range(len(t))))]))
    a['nzeros'] = [0] + (a[1:].MS.values - a[0:len(t)].MS.values + 1).tolist()
    a['ms'] = [''] + map(lambda i: "".join(['0'] * i) + '1', a.nzeros[1:])
    a['runs'] = [-1] + map(lambda s: int(s == '1'), a.ms[1:])
    return a.set_index('i')

def table2(t, s):
    bwt_s, alp, C, sa = bwt(s)

    a = pd.DataFrame(OrderedDict([('i', range(len(s) + 1)), 
                                  ('s_i', list(s + "#")), 
                                  ('BWT', list(bwt_s)),
                                  ('SA', map(lambda i: len(s) - sa[i].tolist().index("#"), range(sa.shape[0]))),
                                  ('suff_SA', map(lambda i: "".join(sa[i][:sa[i].tolist().index("#")+1].tolist()), range(sa.shape[0])))
                                 ]))
    return a.set_index('i')
    

#t = 'ceededdbe'
#s = 'cbeeeddba'
s = 'bcaddcbdaaadccddaacacbcbcbcccccacabbbddddcddcdcaddddacaabdbbdadacacdbbcadcbbcaaccacbabdbcbaaabbbbccadaccdaaaadddabdcbcbcbbcaacdaaadcacaddcdbcdbdbcaccbcaaacaccdabacdaacdacaadbdbbaabdadaccddbabdaacdcdbb'
t = 'acaddcbdaaadccdbaacacbcbcbccccaacabbbddddcddcccaddddacaabdbbaada'
#table2(t, s[::-1]).head()

/Users/denas/arch/Darwin_x86_64/lib/python/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [8]:
tt = table1(t, s)
tt['pre_merge_RUNS'] = [-1] + [0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,1]
tt['error'] = map(int, tt.pre_merge_RUNS != tt.runs)
tt#[tt.error == 1]

,t_i,MS,nzeros,ms,runs,pre_merge_RUNS,error
i,,,,,,,
-1,,1,0,,-1,-1,0
0,a,6,6,0000001,0,0,0
1,c,14,9,0000000001,0,0,0
2,a,13,0,1,1,1,0
3,d,12,0,1,1,1,0
4,d,11,0,1,1,1,0
5,c,10,0,1,1,1,0
6,b,9,0,1,1,1,0
7,d,8,0,1,1,1,0


In [3]:
table2(t, s)#[50:80]

,s_i,BWT,SA,suff_SA
i,,,,
0,b,b,200,#
1,c,d,105,aaaadddabdcbcbcbbcaacdaaadcacaddcdbcdbdbcaccbc...
2,a,b,90,aaabbbbccadaccdaaaadddabdcbcbcbbcaacdaaadcacad...
3,d,c,151,aaacaccdabacdaacdacaadbdbbaabdadaccddbabdaacdc...
4,d,d,127,aaadcacaddcdbcdbdbcaccbcaaacaccdabacdaacdacaad...
5,c,d,8,aaadccddaacacbcbcbcccccacabbbddddcddcdcaddddac...
6,b,a,106,aaadddabdcbcbcbbcaacdaaadcacaddcdbcdbdbcaccbca...
7,d,a,91,aabbbbccadaccdaaaadddabdcbcbcbbcaacdaaadcacadd...
8,a,b,177,aabdadaccddbabdaacdcdbb#


In [4]:
for i, a in enumerate(zip(t, s)):
    print {True: " ", False: "*"}[a[0] == a[1]], i, a

* 0 ('a', 'b')
  1 ('c', 'c')
  2 ('a', 'a')
  3 ('d', 'd')
  4 ('d', 'd')
  5 ('c', 'c')
  6 ('b', 'b')
  7 ('d', 'd')
  8 ('a', 'a')
  9 ('a', 'a')
  10 ('a', 'a')
  11 ('d', 'd')
  12 ('c', 'c')
  13 ('c', 'c')
  14 ('d', 'd')
* 15 ('b', 'd')
  16 ('a', 'a')
  17 ('a', 'a')
  18 ('c', 'c')
  19 ('a', 'a')
  20 ('c', 'c')
  21 ('b', 'b')
  22 ('c', 'c')
  23 ('b', 'b')
  24 ('c', 'c')
  25 ('b', 'b')
  26 ('c', 'c')
  27 ('c', 'c')
  28 ('c', 'c')
  29 ('c', 'c')
* 30 ('a', 'c')
  31 ('a', 'a')
  32 ('c', 'c')
  33 ('a', 'a')
  34 ('b', 'b')
  35 ('b', 'b')
  36 ('b', 'b')
  37 ('d', 'd')
  38 ('d', 'd')
  39 ('d', 'd')
  40 ('d', 'd')
  41 ('c', 'c')
  42 ('d', 'd')
  43 ('d', 'd')
  44 ('c', 'c')
* 45 ('c', 'd')
  46 ('c', 'c')
  47 ('a', 'a')
  48 ('d', 'd')
  49 ('d', 'd')
  50 ('d', 'd')
  51 ('d', 'd')
  52 ('a', 'a')
  53 ('c', 'c')
  54 ('a', 'a')
  55 ('a', 'a')
  56 ('b', 'b')
  57 ('d', 'd')
  58 ('b', 'b')
  59 ('b', 'b')
* 60 ('a', 'd')
  61 ('a', 'a')
  62 ('d', 'd')
  